## Install & Imports

In [1]:
!pip install -q ultralytics

In [2]:
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
from collections import defaultdict

## Image Inference

In [8]:
import cv2
import numpy as np
import os
from glob import glob
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from collections import defaultdict

model = YOLO("model.pt")
names = model.model.names

input_images_folder = "images/"
output_results_folder = "results/"
image_files = glob(os.path.join(input_images_folder, "*.jpg"))

# Ensure the output folder exists
os.makedirs(output_results_folder, exist_ok=True)

# Initialize variables
confidence_threshold = 0.7
coin_value_mapping = {0: 0.50, 1: 0.25, 2: 1, 3: 0.10, 4: 10, 5: 20, 6: 0.05, 7: 5, 8: 50}

# Process each image in the folder
for image_file in image_files:

    coin_counts = defaultdict(int)

    image = cv2.imread(image_file)
    results = model(image)
    boxes = results[0].boxes.xyxy.cpu()

    # Annotator Init
    annotator = Annotator(image, line_width=2)

    # Process detected objects
    for box, cls, conf in zip(boxes, results[0].boxes.cls.cpu().tolist(), results[0].boxes.conf.float().cpu().tolist()):
        # Filter out predictions below the confidence threshold
        if conf >= confidence_threshold:
            annotator.box_label(box, color=colors(int(cls), True), label=f"{names[int(cls)]} {conf:.2f}")
            coin_counts[int(cls)] += 1

    # Calculate total value and update counter
    counter = len(boxes)
    total_value = sum(coin_value_mapping[coin_type] * count for coin_type, count in coin_counts.items())

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_size = 1
    font_thickness = 2

    cv2.putText(image, f'Number of Objects: {counter}', (10, 50), font, font_size, (255, 0, 0), font_thickness, cv2.LINE_AA)
    cv2.putText(image, f'Total Value: {total_value:.2f} JD', (10, 20), font, font_size, (255, 0, 0), font_thickness, cv2.LINE_AA)

    # Save the result image in the output folder
    result_image_path = os.path.join(output_results_folder, os.path.basename(image_file))
    cv2.imwrite(result_image_path, image)



0: 640x640 2 1-4-dinars, 1 10-Piastres, 2 10-dinars, 1 20-dinar, 2 5-Piastress, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 1-2-dinar, 2 1-dinars, 3 10-Piastress, 1 10-dinar, 1 5-Piastres, 2 5-dinars, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 640x640 1 1-2-dinar, 1 1-4-dinar, 2 1-dinars, 2 50-dinars, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 20-dinar, 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


## Video Inference

In [9]:
import cv2
from glob import glob
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from collections import defaultdict

model = YOLO("model.pt")
names = model.model.names

input_video_path = "videos/1.mp4"
output_video_path = "results/1-result.mp4"
cap = cv2.VideoCapture(input_video_path)
assert cap.isOpened(), "Error reading video file"

# Initialize variables
font = cv2.FONT_HERSHEY_SIMPLEX
counter = 0
coin_counts = defaultdict(int)
confidence_threshold = 0.7
coin_value_mapping = {0: 0.50, 1: 0.25, 2: 1, 3: 0.10, 4: 10, 5: 20, 6: 0.05, 7: 5, 8: 50}

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create VideoWriter object for output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()

    if not ret:
        break

    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu()

    # Annotator Init
    annotator = Annotator(frame, line_width=2)

    # Process detected objects
    for box, cls, conf in zip(boxes, results[0].boxes.cls.cpu().tolist(), results[0].boxes.conf.float().cpu().tolist()):
        # Filter out predictions below the confidence threshold
        if conf >= confidence_threshold:
            annotator.box_label(box, color=colors(int(cls), True), label=f"{names[int(cls)]} {conf:.2f}")

            # Update coin counts
            coin_counts[int(cls)] += 1

    # Calculate total value and update counter
    counter = len(boxes)  # Count the number of detected objects
    total_value = sum(coin_value_mapping[coin_type] * count for coin_type, count in coin_counts.items())

    font_size = 1.1
    font_thickness = 2
    # Specify the font explicitly
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, f'Number of Objects: {counter}', (10, 50), font, font_size, (255, 0, 0), font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'Total Value: {total_value:.2f} JD', (10, 20), font, font_size, (255, 0, 0), font_thickness, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()


0: 640x384 1 1-2-dinar, 2 10-Piastress, 2 5-Piastress, 2 50-dinars, 136.0ms
Speed: 2.0ms preprocess, 136.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 1-2-dinar, 2 10-Piastress, 2 5-Piastress, 2 50-dinars, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 1-2-dinar, 2 10-Piastress, 2 5-Piastress, 2 50-dinars, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 1-2-dinar, 2 10-Piastress, 2 5-Piastress, 2 50-dinars, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 1-2-dinar, 2 10-Piastress, 1 5-Piastres, 2 50-dinars, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 1-2-dinar, 2 10-Piastress, 1 5-Piastres, 2 50-dinars, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 2.0ms postproces